In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE, RandomOverSampler

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_data = pd.read_csv('NSL-KDD-Dataset/KDDTrain+.csv')
test_data = pd.read_csv('NSL-KDD-Dataset/KDDTest+.csv')
#train_data[train_data['service']!='-']
cols_to_transform = ['protocol_type', 'services', 'flag']
#train_data = pd.get_dummies(data = train_data, columns = cols_to_transform)
#test_data = pd.get_dummies(data = test_data, columns = cols_to_transform)
#train_data[['label','attack_cat']][train_data['attack_cat']!='Dos'].head(10000)
#obj_df = train_data.select_dtypes(include=['object']).copy()
for col in cols_to_transform:
    train_data[col]= train_data[col].astype('category').cat.codes
    test_data[col]= test_data[col].astype('category').cat.codes
train_data_ = train_data.drop(['num_outbound_cmds','class_label'], axis=1)
test_data_ = test_data.drop(['num_outbound_cmds','class_label'], axis=1)
train_data_ = train_data_.as_matrix(columns = None)
test_data_ = test_data_.as_matrix(columns = None)
test_data_.shape


(22545, 41)

In [3]:
x_train= train_data_[:,:-1]
y_train = train_data_[:,-1]
x_test = test_data_[:,:-1]
y_test = test_data_[:,-1]
x_min = []
x_max = []
for i in range(x_train.shape[1]):
    x_min.append(np.min(x_train[:,i]))
    x_max.append(np.max(x_train[:,i]))
x_max = np.array(x_max)
x_min = np.array(x_min)
x_train = (x_train - x_min)/(x_max-x_min)
x_test = (x_test - x_min)/(x_max-x_min)
x_test = x_test[:-1]
y_test = y_test[:-1]

In [36]:
# Number of Epochs
dim_size = 30
epochs = 200
num_seq = 1
# Batch Size
batch_size =200
# RNN Size
rnn_size =50
# Learning Rate
learning_rate = 0.0005
num_layers = 1
# Show stats for every n number of batches
show_every_n_batches = 100
keep_prob = 0.5
relu_size = 50
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

In [37]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = dim_size)
x_reduced = svd.fit_transform(x_train)
x_reduced[:10]

array([[ 1.32236566e+00, -2.28826003e-01,  9.68252263e-02,
         4.86907317e-01, -3.37092847e-01,  3.10851823e-01,
        -2.20936845e-01,  1.84154175e-01, -3.05602369e-01,
         1.16690320e-01,  8.43966141e-02, -1.07548648e-01,
        -1.61493224e-01,  4.74326274e-02,  1.16950327e-01,
        -1.34933274e-01, -2.40885055e-02, -9.48888487e-02,
        -3.53538221e-02, -9.47714734e-02,  7.66154260e-02,
        -9.05532430e-02,  2.28095738e-03, -1.34439430e-02,
         1.56069131e-03, -2.06610855e-02, -3.22240112e-04,
        -7.58615677e-04,  2.53775065e-03,  3.40515422e-04],
       [ 1.35046069e+00,  2.59793799e-01,  4.35092859e-01,
         7.80733812e-01, -6.45491350e-01,  7.52956059e-01,
        -9.14937032e-02,  1.68794485e-02,  4.23706159e-01,
        -3.87287405e-01, -1.98567432e-01,  4.07497663e-01,
         2.55590568e-01,  3.77393376e-02, -3.81232140e-02,
         9.42956878e-03,  2.12495512e-01, -1.80928062e-02,
        -1.43276516e-02, -7.25561463e-02, -2.22065042e-

In [38]:
x_test_reduced = svd.transform(x_test)
x_train.shape

(125973, 40)

In [39]:
def resample(arr,y, num_seq):
    
    new_arr = np.zeros((arr.shape[0],num_seq,arr.shape[1]))
    new_y = np.zeros(arr.shape[0])
    for i in range(arr.shape[0]-num_seq):
        for j in range(num_seq):
            new_arr[i,j,:] = arr[i+j,:]
        new_y[i] = y[i+num_seq-1]
    x_resampled ,y_resampled = SMOTE().fit_sample(np.reshape(new_arr,[-1,num_seq*arr.shape[1]]), new_y)
    x_resampled = np.reshape(x_resampled,[-1,num_seq,arr.shape[1]])
    return x_resampled,y_resampled
    #return new_arr,new_y

In [40]:
x_resampled = []
y_resampled = []
for i in range(y_train.size//1000):
    x_resampledi, y_resampledi = resample(x_reduced[i*1000:1000*(i+1)], y_train[i*1000:1000*(i+1)],num_seq)
    x_resampled.append(x_resampledi)
    y_resampled.append(y_resampledi)
#x_resampledj,y_resampledj = resample(x_reduced[75000:], y_train[75000:],num_seq)
#x_resampled.append(x_resampledj)
#y_resampled.append(y_resampledj)
x_resampled = np.array(np.concatenate(x_resampled, axis = 0))    
y_resampled = np.array(np.concatenate(y_resampled, axis = 0))
from random import shuffle
ind_list = [i for i in range(x_resampled.shape[0])]
shuffle(ind_list)
x_train_ = x_resampled[ind_list]
y_train_ = y_resampled[ind_list]

In [41]:
y_train_[:10]

array([1., 0., 1., 0., 1., 1., 1., 1., 0., 0.])

In [42]:
x_valid = x_train_[-40000:]
y_valid = y_train_[-40000:]
train_x = x_train_[:-40000]
train_y = y_train_[:-40000]
x_train.shape
np.mean(train_y)
train_y

array([1., 0., 1., ..., 1., 1., 0.])

In [43]:
def get_batches(arr,y, batch_size):
    n_batches = arr.shape[0]//batch_size
    for i in range(0,n_batches*batch_size,batch_size):
        x = arr[i:i+batch_size,:,:]
        y_ = y[i:i+batch_size]
        yield x,y_
x,y = next(get_batches(train_x, train_y, batch_size))
y.shape

(200,)

In [44]:
def get_test_batches(arr,y, batch_size, num_seq=num_seq):
    new_arr = np.zeros((arr.shape[0],num_seq,arr.shape[1]))
    new_y = np.zeros(arr.shape[0])
    for i in range(arr.shape[0]-num_seq):
        for j in range(num_seq):
            new_arr[i,j,:] = arr[i+j,:]
        new_y[i] = y[i+num_seq-1]
    n_batches = arr.shape[0]//batch_size
    for i in range(0,n_batches*batch_size,batch_size):
        x = new_arr[i:i+batch_size,:,:]
        y = new_y[i:i+batch_size]
        yield x,y
next(get_test_batches(x_test_reduced, y_test,100))       

(array([[[ 1.01293012e+00,  4.35494664e-01,  2.09369311e+00, ...,
           3.49895539e-04,  1.06489316e-03, -5.03614168e-05]],
 
        [[ 9.62122067e-01,  4.40164641e-01,  2.08573104e+00, ...,
           5.94780357e-04,  1.19269372e-03, -9.71355139e-05]],
 
        [[ 1.53995130e+00, -4.51072468e-01,  2.55773964e-02, ...,
           1.11691458e-03,  1.24406357e-03, -1.02300494e-04]],
 
        ...,
 
        [[ 2.13825193e+00, -5.76708062e-01, -4.03953467e-03, ...,
           3.87577404e-04,  2.64297080e-04, -1.27516369e-04]],
 
        [[ 1.86204993e+00,  1.29133525e+00, -3.34787269e-01, ...,
           2.87780141e-03,  3.02120155e-03,  1.59356700e-03]],
 
        [[ 1.09115856e+00,  4.25425984e-01,  2.09388678e+00, ...,
           4.67406576e-04,  2.06359524e-03, -1.52924506e-04]]]),
 array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,

In [45]:
def build_inputs(batch_size, num_inputs = 100 ):
    inputs = tf.placeholder(tf.float32, [None,num_inputs,dim_size], name = 'inputs')
    outputs = tf.placeholder(tf.int32, [None,],name = 'targets')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    return inputs, outputs, keep_prob

In [46]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    return cell, initial_state   

In [47]:
def build_output(lstm_output, in_size, out_size, relu_size = 100):
    out = tf.contrib.layers.fully_connected(lstm_output, relu_size, activation_fn = tf.nn.relu)
    out = tf.layers.batch_normalization(out)
    logits = tf.contrib.layers.fully_connected(out,out_size, activation_fn=None)
    #logits = tf.reshape(logits, (-1,10,3, out_size))
    return logits


In [48]:
def build_loss(logits, targets,vocab_size):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
      # One-hot encode targets and reshape to match logits, one row per batch_size per step
    #logits = tf.reshape(logits,[-1,vocab_size])
    y_one_hot = tf.one_hot(targets, vocab_size)
    #y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    #y_reshaped = tf.reduce_sum(y_one_hot, axis = 1)
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_one_hot)
    loss = tf.reduce_mean(loss)
    return loss, y_one_hot

In [49]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [50]:
class RNN:
    
    def __init__(self, vocab_size, relu_size, batch_size=64,
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_inputs = num_seq )

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
       
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, self.inputs, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.logits = build_output(outputs, lstm_size, vocab_size, relu_size = relu_size)[:,-1]
        
        # Loss and optimizer (with gradient clipping)
        self.loss, self.y_reshaped = build_loss(self.logits, self.targets, vocab_size)
        self.pred = tf.argmax(self.logits,1)
        #_,self.pred = tf.nn.top_k(self.logits,3)
        correct_pred = tf.equal(self.pred, tf.argmax(self.y_reshaped,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [51]:
save_every_n = 200

model = RNN(2,relu_size, batch_size=batch_size,
                lstm_size=rnn_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        train_loss = []
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(train_x, train_y,batch_size):
            
            counter += 1
            
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            train_loss.append(batch_loss)
            #if(counter%show_every_n_batches ==0):
                #print('Epoch: {}/{}... '.format(e+1, epochs),
                  #'Training Step: {}... '.format(counter),
                    #'Training loss: {:.4f}... '.format(batch_loss))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, num_layers))
        val_loss = []
        val_state = sess.run(model.initial_state)
        for x, y in get_batches(x_valid, y_valid, batch_size):
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: val_state}
            batch_loss, val_state = sess.run([model.loss, 
                                                 model.final_state], 
                                                 feed_dict=feed)
            
                                                 
            val_loss.append(batch_loss)
            
        print("epoch {}: Training loss : {:.6f}".format(e, np.mean(train_loss)))
        print("Validation loss : {:.6f}".format(np.mean(val_loss)))
        
    val_acc = []
    val_pred =[]
    val_y = []
    val_logits = []
    val_state = sess.run(model.initial_state)
    for x, y in get_batches(x_valid, y_valid,batch_size):
        feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: val_state}
        val_state,batch_pred,batch_acc,batch_logits = sess.run([model.final_state,model.pred,model.accuracy, model.logits], feed_dict = feed)
        val_acc.append(batch_acc)
        val_y.append(y)
        val_pred.append(batch_pred)
        val_logits.append(batch_logits)
    train_pred =[]
    train_label = []
    train_acc = []
    train_logits = []
    train_state = sess.run(model.initial_state)
    for x, y in get_batches(train_x, train_y,batch_size):
        feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: train_state}
        train_state,batch_pred,batch_acc, batch_logits = sess.run([model.final_state,model.pred,model.accuracy, model.logits], feed_dict = feed)
        train_acc.append(batch_acc)
        train_label.append(y)
        train_pred.append(batch_pred)
        train_logits.append(batch_logits)
    print("Training accuracy : {:.2f}".format(100*np.mean(train_acc)))    
    print("validation accuracy : {:.2f}".format(100*np.mean(val_acc)))
    test_acc = []
    test_pred =[]
    test_y = []
    test_logits = []
    test_state = sess.run(model.initial_state)
    for x, y in get_test_batches(x_test_reduced, y_test,batch_size,num_seq):
        feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: val_state}
        test_state,batch_pred,batch_acc, batch_logits = sess.run([model.final_state,model.pred,model.accuracy,model.logits], feed_dict = feed)
        test_acc.append(batch_acc)
        test_y.append(y)
        test_pred.append(batch_pred)
        test_logits.append(batch_logits)
    print("Test accuracy : {:.2f}".format(100*np.mean(test_acc))) 
    #val_pred = np.vstack(val_pred)
    #val_y = np.vstack(val_y)
    #print(mapk(val_y,val_pred))
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, num_layers))

epoch 0: Training loss : 0.174829
Validation loss : 0.057415
epoch 1: Training loss : 0.051725
Validation loss : 0.039678
epoch 2: Training loss : 0.040569
Validation loss : 0.030937
epoch 3: Training loss : 0.033897
Validation loss : 0.024985
epoch 4: Training loss : 0.027858
Validation loss : 0.020762
epoch 5: Training loss : 0.024964
Validation loss : 0.017981
epoch 6: Training loss : 0.021313
Validation loss : 0.015026
epoch 7: Training loss : 0.018916
Validation loss : 0.012329
epoch 8: Training loss : 0.016096
Validation loss : 0.010338
epoch 9: Training loss : 0.014115
Validation loss : 0.009010
epoch 10: Training loss : 0.012723
Validation loss : 0.007875
epoch 11: Training loss : 0.011675
Validation loss : 0.006913
epoch 12: Training loss : 0.010406
Validation loss : 0.006475
epoch 13: Training loss : 0.009526
Validation loss : 0.005756
epoch 14: Training loss : 0.008655
Validation loss : 0.005251
epoch 15: Training loss : 0.007904
Validation loss : 0.004933
epoch 16: Training

epoch 132: Training loss : 0.001600
Validation loss : 0.004045
epoch 133: Training loss : 0.001761
Validation loss : 0.004635
epoch 134: Training loss : 0.001845
Validation loss : 0.004451
epoch 135: Training loss : 0.001807
Validation loss : 0.004291
epoch 136: Training loss : 0.001686
Validation loss : 0.003953
epoch 137: Training loss : 0.001634
Validation loss : 0.004138
epoch 138: Training loss : 0.001779
Validation loss : 0.004283
epoch 139: Training loss : 0.001714
Validation loss : 0.004339
epoch 140: Training loss : 0.001661
Validation loss : 0.004900
epoch 141: Training loss : 0.001881
Validation loss : 0.004330
epoch 142: Training loss : 0.001560
Validation loss : 0.004280
epoch 143: Training loss : 0.001587
Validation loss : 0.004353
epoch 144: Training loss : 0.001524
Validation loss : 0.004520
epoch 145: Training loss : 0.001670
Validation loss : 0.004615
epoch 146: Training loss : 0.001774
Validation loss : 0.004217
epoch 147: Training loss : 0.001457
Validation loss : 0

In [60]:
y_ =[]
print(np.sum([np.sum(pred)for pred in val_pred]), np.sum([np.sum(pred)for pred in val_y]))
print(np.sum([np.sum(pred) for pred in test_pred]))

20112 20113.0
6272


In [62]:
val_pred


array([1, 1, 1, ..., 0, 1, 1], dtype=int64)

In [63]:
val_y 

array([1., 1., 1., ..., 0., 1., 1.])

In [64]:
from sklearn.metrics import confusion_matrix
tn,fp,fn,tp = confusion_matrix(np.concatenate(test_y),np.concatenate(test_pred)).ravel()
(tp,fp,fn,tn)

(5562, 710, 145, 15983)

In [65]:
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,np.mean(test_acc))

(0.8867984693877551,
 0.042532798178877375,
 0.9745926055721045,
 0.9286250939143501,
 0.96183044)

In [57]:
y_train[75000:]

array([0., 0., 0., ..., 0., 1., 0.])

In [58]:
train_x.shape

(118940, 1, 30)

In [66]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(np.reshape(train_x,[-1,30]),train_y)
val_pred = clf.predict(np.reshape(x_valid,[-1,30]))
test_pred = clf.predict(x_test_reduced)

In [67]:
from sklearn.metrics import confusion_matrix
tn,fp,fn,tp = confusion_matrix(y_valid,val_pred).ravel()
(tp,fp,fn,tp)

(18429, 8805, 1684, 18429)

In [68]:
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)

(0.6766909010795329,
 0.44275154623623475,
 0.9162730572266693,
 0.778465372674087,
 0.737775)

In [87]:
tn,fp,fn,tp = confusion_matrix(np.concatenate(train_label,0),np.concatenate(train_pred,0)).ravel()
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)

(0.999932505399568,
 6.720317198971792e-05,
 0.9996794817726344,
 0.9998059775777565,
 0.9998063973063973)

In [78]:
train_logits
clf.fit(np.concatenate(train_logits,0),train_y[:118800])

GaussianNB(priors=None)

In [79]:
val_pred = clf.predict(np.concatenate(val_logits,0))

In [81]:
tn,fp,fn,tp = confusion_matrix(y_valid,val_pred).ravel()
(tp,fp,fn,tp)

(19606, 2, 507, 19606)

In [82]:
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)

(0.9998980008159934,
 0.00010056821038869613,
 0.9747924228111172,
 0.9871856196973894,
 0.987275)

In [92]:
test_pred_ = clf.predict(np.concatenate(test_logits,0))
tn,fp,fn,tp = confusion_matrix(np.concatenate(test_y,0),test_pred_).ravel()
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)

(0.9643122676579926,
 0.011501827113161205,
 0.9090590502891186,
 0.9358708397221971,
 0.9682589285714286)